# Install Dependencies


In [ ]:
# Install dependencies
!pip install -q sentence-transformers pandas torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.3 MB/s eta 0:00:00


In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.2 MB/s eta 0:00:00


In [ ]:
# Imports
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
import numpy as np
import os

# Import Job Descriptions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/University/Machine Learning/Mini Project/Model/job_data.csv'

# Load the dataset
job_data = pd.read_csv(file_path)

# Preview
# job_data.head()

# Load Pre-trained Model and Save Embeddings

In [ ]:
# Load pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
job_embeddings = model.encode(job_data['description'].tolist(), convert_to_tensor=False)

In [ ]:
# Save embeddings and CSV
save_path = '/content/drive/MyDrive/University/Machine Learning/Mini Project/Model'
os.makedirs(save_path, exist_ok=True)

np.save(os.path.join(save_path, 'job_embeddings.npy'), job_embeddings)
job_data.to_csv(os.path.join(save_path, 'job_data.csv'), index=False)

In [ ]:
# Mount again
drive.mount('/content/drive')

# Load saved files
job_embeddings = np.load('/content/drive/MyDrive/University/Machine Learning/Mini Project/Model/job_embeddings.npy')
job_data = pd.read_csv('/content/drive/MyDrive/University/Machine Learning/Mini Project/Model/job_data.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import User Profile

In [ ]:
import fitz  # PyMuPDF

pdf_path = "/content/drive/MyDrive/University/Machine Learning/Mini Project/Model/profiles/profile_1.pdf"
doc = fitz.open(pdf_path)

user_profile = ""
for page in doc:
    user_profile += page.get_text()

# print(user_profile)

In [ ]:
user_profile = user_profile.strip().replace('\n', ' ')

# print(user_profile)

In [ ]:
# Encode user input
user_embedding = model.encode(user_profile, convert_to_tensor=False)

# Compute Similarity

## Top 5 without duplicate titles

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Compute similarity
similarities = cosine_similarity([user_embedding], job_embeddings)[0]
top_indices = similarities.argsort()[::-1]  # Sort all, we'll filter duplicates next

# Build unique recommendation list
seen_titles = set()
recommendations = []

for i in top_indices:
    title = job_data["title"][i]
    if title not in seen_titles:
        seen_titles.add(title)
        recommendations.append({
            "Job Title": title,
            "Match Score": round(similarities[i], 4)
        })
    if len(recommendations) >= 5:
        break

# Display top 10 unique-title recommendations
pd.DataFrame(recommendations)

,Job Title,Match Score
0,Software Engineer,0.5604
1,Machine Learning,0.4764
2,DevOps Engineer,0.4718
3,JavaScript Developer,0.4616
4,Network Administrator,0.4615


## Top 10

In [ ]:
# Compute similarity
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity([user_embedding], job_embeddings)[0]
top_indices = similarities.argsort()[::-1][:10]

# Build recommendation list
recommendations = [
    {
        "Job Title": job_data["title"][i],
        "Match Score": round(similarities[i], 4)
    }
    for i in top_indices
]

# Display recommendations
pd.DataFrame(recommendations)

,Job Title,Match Score
0,Software Engineer,0.5604
1,Software Engineer,0.4925
2,Machine Learning,0.4764
3,DevOps Engineer,0.4718
4,Software Engineer,0.4639
5,JavaScript Developer,0.4616
6,Network Administrator,0.4615
7,DevOps Engineer,0.4592
8,Machine Learning,0.4591
9,JavaScript Developer,0.4566


---